<a href="https://colab.research.google.com/github/GrandPi224/pradnya-site/blob/main/Yahoo_Bulk_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install libraries {"display-mode":"form"}
!pip install yfinance
!pip install --upgrade yfinance
!pip install pandas
!pip install openpyxl
from google.colab import data_table
from vega_datasets import data

import yfinance as yf
from datetime import datetime
import pandas as pd
from google.colab import files
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# @title Get Stock Data From Yahoo {"display-mode":"form"}

from google.colab import data_table
data_table.enable_dataframe_formatter()


# Create input widgets
ticker_input = widgets.Text(
    value='ADM, CMCSA',
    placeholder='Enter ticker symbols separated by commas (e.g., AAPL, MSFT)',
    description='Comma Separated Tickers:',
    layout={'width': '400px'}
)

start_date_input = widgets.Text(
    value='2024-01-01',
    placeholder='YYYY-MM-DD',
    description='Start Date:',
    layout={'width': '300px'}
)

end_date_input = widgets.Text(
    value=datetime.now().strftime('%Y-%m-%d'),
    placeholder='YYYY-MM-DD',
    description='End Date:',
    layout={'width': '300px'}
)

attributes_input = widgets.SelectMultiple(
    options=['Daily_Return', 'Dividends','Daily_Dividend_Yield','MA20', 'MA50', 'Volume', 'MACD', 'Volatility', 'Price_Change'],
    value=['Daily_Return'],
    description='Additional Attributes:',
    layout={'width': '400px', 'height': '150px'},
)

adjusted_dropdown = widgets.Dropdown(
    options=[('Use Adjusted Prices', True), ('Use Raw Prices (Close)', False)],
    value=True,
    description='Price Type:',
    layout={'width': '400px'}
)

file_format_input = widgets.Dropdown(
    options=['csv', 'xlsx'],
    value='csv',
    description='File Format:',
    layout={'width': '300px'}
)

fetch_button = widgets.Button(
    description='Fetch Data',
    button_style='success',
    tooltip='Click to fetch stock data'
)

output = widgets.Output()

# Flag to prevent multiple simultaneous executions
is_fetching = False

def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False

#Helper functions additional attributes
def calculate_macd(data, fast=12, slow=26, signal=9):
    ema_fast = data['Close'].ewm(span=fast, adjust=False).mean()
    ema_slow = data['Close'].ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line


def calculate_volatility(data, window=20):
    returns = data['Close'].pct_change()
    return returns.rolling(window=window).std() * (252 ** 0.5)

def calculate_price_change(data):
    return data['Close'].diff()


def calculate_dividend_yield(data):
    data['Date'] = pd.to_datetime(data['Date'])
    data['Year'] = data['Date'].dt.year

    # Only rows with actual dividend payments
    dividend_data = data[data['Dividends'] > 0]

    if dividend_data.empty:
        return pd.DataFrame(columns=['Year', 'Ticker', 'Payout_Date', 'Payout'])

    # Get latest dividend per year and ticker
    latest_dividends = (
        dividend_data.sort_values(by='Date')
        .groupby(['Year', 'Ticker'], group_keys=False)
        .apply(lambda x: x.loc[x['Date'].idxmax()])
        .reset_index(drop=True)
        .rename(columns={'Date': 'Payout_Date', 'Dividends': 'Payout'})
    )

    return latest_dividends[['Year', 'Ticker', 'Payout_Date', 'Payout']]

# Fetch and process data with pretty-printed DataFrame
def fetch_stock_data(b):
    global is_fetching
    with output:
        if is_fetching:  # Prevent multiple executions
            print("Fetching already in progress, please wait...")
            return

        is_fetching = True
        output.clear_output()

        tickers = [t.strip().upper() for t in ticker_input.value.split(',') if t.strip()]
        start_date = start_date_input.value
        end_date = end_date_input.value
        attributes = list(attributes_input.value)
        file_format = file_format_input.value

        if not tickers or not (is_valid_date(start_date) and is_valid_date(end_date)):
            print("Please enter valid ticker(s) and dates in YYYY-MM-DD format")
            is_fetching = False
            return

        try:
            all_data = {}
            for ticker in tickers:
                stock = yf.Ticker(ticker)
                df = stock.history(start=start_date, end=end_date, interval="1d",auto_adjust=adjusted_dropdown.value)

                if df.empty:
                    print(f"No data found for {ticker}")
                    continue

                # Normalize dates to remove time component
                df = df.reset_index()

                df['Ticker'] = ticker


                # Add attributes

                if 'Daily_Return' in attributes:
                    df['Daily_Return'] = df['Close'].pct_change()
                if 'MA20' in attributes:
                    df['MA20'] = df['Close'].rolling(window=20).mean()
                if 'MA50' in attributes:
                    df['MA50'] = df['Close'].rolling(window=50).mean()
                if 'MACD' in attributes:
                    df['MACD'], df['MACD_Signal'] = calculate_macd(df)
                if 'Volatility' in attributes:
                    df['Volatility'] = calculate_volatility(df)
                if 'Price_Change' in attributes:
                    df['Price_Change'] = calculate_price_change(df)
                if 'Volume' not in attributes:
                    df = df.drop(columns=['Volume'], errors='ignore')
                if 'Dividends' not in attributes and 'Daily_Dividend_Yield' not in attributes:
                    df = df.drop(columns=['Dividends'], errors='ignore')
                if 'Daily_Dividend_Yield' in attributes:
                    dividend_df = calculate_dividend_yield(df)
                    if not dividend_df.empty:
                        df['Year'] = df['Date'].dt.year
                        # Merge on both Year and Ticker
                        merged_data = pd.merge(df, dividend_df, on=['Year', 'Ticker'], how='left')

                        # Annualize the latest dividend (assuming quarterly)
                        merged_data['Annualized_Dividend'] = merged_data['Payout'] * 4

                        # Calculate daily dividend yield
                        merged_data['Daily_Dividend_Yield'] = merged_data['Annualized_Dividend'] / merged_data['Close']

                        # Inject back into df
                        df['Daily_Dividend_Yield'] = merged_data['Daily_Dividend_Yield']
                        df = df.loc[:, ~df.columns.duplicated()]
                    else:
                        df['Daily_Dividend_Yield'] = "dividend yield was empty"


                all_data[ticker] = df

            if not all_data:
                print("No data retrieved")
                is_fetching = False
                return

            combined_df = pd.concat(all_data.values(), axis=0)
            combined_df['Date'] = pd.to_datetime(combined_df['Date']).dt.strftime('%m-%d-%Y')

            # Drop the 'Year' column if it exists
            combined_df = combined_df.drop(columns=['Year'], errors='ignore')

            # Round numeric values for readability
            round_columns = ['Open', 'High', 'Low', 'Close', 'Dividends', 'Daily_Return','MA20', 'MA50', 'MACD', 'MACD_Signal',
            'Volatility', 'Price_Change', 'Daily_Dividend_Yield']

            # Only round columns that exist in the current DataFrame
            for col in round_columns:
                if col in combined_df.columns:
                    combined_df[col] = combined_df[col].round(2)

            # Pretty print the DataFrame
            print(f"Combined Data for {', '.join(tickers)} (First 10 rows):")
            display(combined_df.head(10))

            # File saving based on user-selected format
            base_filename = f"multi_ticker_stock_data_{start_date}_to_{end_date}"
            if file_format == 'csv':
                filename = f"{base_filename}.csv"
                combined_df.to_csv(filename, index=True)
                print(f"Saving {filename}...")
                files.download(filename)
                print(f"\nData saved and downloaded as {filename}")
            elif file_format == 'xlsx':
                filename = f"{base_filename}.xlsx"
                combined_df.to_excel(filename, index=True, engine='openpyxl')
                print(f"Saving {filename}...")
                files.download(filename)
                print(f"\nData saved and downloaded as {filename}")

        except Exception as e:
            print(f"Error: {str(e)}")

        finally:
            is_fetching = False  # Reset flag after execution

fetch_button.on_click(fetch_stock_data)
# Define Bold Labels using HTML
title_label = widgets.HTML("<b style='color:blue;'>Multi-Ticker Stock Data Fetcher.</b>")
fields_label = widgets.HTML("<b style='color:blue;'>Default fields include: Date, Open, High, Low, Close.</b>")


display(widgets.VBox([
    title_label,
    fields_label,
    ticker_input,
    start_date_input,
    end_date_input,
    attributes_input,
    adjusted_dropdown,
    file_format_input,
    fetch_button,
    output
]))




NameError: name 'widgets' is not defined